<a href="https://colab.research.google.com/github/LinarKulinar/ml-for-sec-lab2/blob/main/Get_feature_from_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import random
from random import random, randrange, randint

from operator import itemgetter

import os, sys
import pandas as pd


In [2]:
# dir_name = 'BOWS2/'
dir_name = 'BOWS2_TMP/'
pictures = os.listdir(dir_name)
count_pictures = len(pictures)

imgs_raw = []
for k in range(count_pictures):
  im = cv2.imread(f'{dir_name}{pictures[k]}', 0)
  imgs_raw.append(im)

In [3]:
imgs = np.array(imgs_raw)

#### Опишем функцию `get_features`, которая из изображения возвращает метод признаков, с помощью метода пар значений.

In [4]:
q = 2 # Вторая битовая плоскость

def get_features(img, method, q_beatmap = 2):
  """
      Метод вычленяет вектор признаков из изображений
      :param img: Исходное изображение
      :param method: Имя метода, которым будет совершаться преобразование изображения в вектор признаков
      :param q_beatmap: номер битовой плоскости, в которой происходит поиск
      """
  assert len(img.shape) == 2

  def get_features_pixel_pairs_value(image, q_beatmap = 2):
    """
      Вычленяет вектор признаков методом пар значений.
      Метод основывается на гипотезе о том, 
      что равновероятно количество яркостей пикселя, 
      отличающихся только битом на битовой плоскости q_beatmap
      :param img: Исходное изображение
      :param q_beatmap: номер битовой плоскости, в которой происходит поиск
      """
    
    def get_hist_theoretical(hist_empiritical, q_beatmap = 2):
      """
      Вычисляет теоретическую гистограмму по эмпирической гистограмме 
      на основе гипотезы о том, что равновероятно количество яркостей пикселя, 
      отличающихся только битом на битовой плоскости q_beatmap
      :param hist_empiritical: эмпирическая гистограмма изображения
      :param q_beatmap: номер битовой плоскости, в которой происходит поиск
      """
      assert 1 <= q_beatmap <= 8, "q_beatmap отличается от допустимых значений"
      assert len(hist_empiritical) == 256
      assert len(hist_empiritical) > 2 ** (q_beatmap - 1), "q_beatmap больше значения возможного для hist_e"
      bitmask = 1 << (q_beatmap - 1) # битмаска с единицей на месте q_beatmap
      bitmask_inverse = 255 - (bitmask)
      result = []
      for i in range(len(hist_empiritical)):
        index1 = i & bitmask_inverse  # обнулили бит на битовой плоскости q_beatmap
        index2 = i | bitmask # поставили единицу на бит на битовой плоскости q_beatmap
        result.append((hist_empiritical[index1] + hist_empiritical[index2]) / 2)
      return np.array(result)

    hist_empiritical = np.histogram(image, bins=256, range=(0, 256))[0]
    hist_theoretical = get_hist_theoretical(hist_empiritical, q)
    return (np.array(hist_empiritical) - hist_theoretical) ** 2

  if method == 'pixel_pairs_value':
    return get_features_pixel_pairs_value(img)
  else:
    raise ValueError("Invalid param 'method' value")
  


# get_features(imgs[0], 'pixel_pairs_value', q_beatmap=2)

array([0.00000000e+00, 1.56025000e+03, 0.00000000e+00, 1.56025000e+03,
       1.71396000e+05, 2.69880250e+05, 1.71396000e+05, 2.69880250e+05,
       8.10000000e+01, 4.90000000e+01, 8.10000000e+01, 4.90000000e+01,
       6.15040000e+04, 4.55625000e+03, 6.15040000e+04, 4.55625000e+03,
       2.65225000e+03, 2.31040000e+04, 2.65225000e+03, 2.31040000e+04,
       6.32025000e+03, 1.87690000e+04, 6.32025000e+03, 1.87690000e+04,
       1.69000000e+02, 2.72250000e+02, 1.69000000e+02, 2.72250000e+02,
       1.22500000e+01, 2.31040000e+04, 1.22500000e+01, 2.31040000e+04,
       7.22500000e+03, 9.60400000e+03, 7.22500000e+03, 9.60400000e+03,
       2.03062500e+04, 1.32250000e+04, 2.03062500e+04, 1.32250000e+04,
       1.89225000e+03, 2.75625000e+03, 1.89225000e+03, 2.75625000e+03,
       1.22500000e+01, 6.76000000e+02, 1.22500000e+01, 6.76000000e+02,
       2.02500000e+01, 1.00000000e+02, 2.02500000e+01, 1.00000000e+02,
       1.80625000e+03, 2.50000000e-01, 1.80625000e+03, 2.50000000e-01,
      

### Передадим изображения в метод `get_features`

In [6]:
features = []
for img in imgs:
  features.append(get_features(img, 'pixel_pairs_value', q_beatmap=2))
features = np.array(features)

In [9]:
features.shape

(5, 256)

In [10]:
# todo Надо не забыть отнормиовать фичи